In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

@njit
def diffeq_accuracy(dy, t, y):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    
@njit
def diffeq_accuracy_2(t, y, dy):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)

@njit
def diffeq_accuracy_3(t, y):
    dy = np.empty(y.shape, dtype=np.float64)
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    return dy

@njit
def correct_answer(t, c1_, c2_):
    y = np.empty((2, t.size), dtype=np.float64)
    y[0] = -c1_ * np.sin(t) + c2_ * np.cos(t) - (np.cos(t) / 2)  # -c1 * sin(t) + c2 * cos(t) - cos(t) / 2
    # At t=0; y = c2 - 1/2
    y[1] = c2_ * np.sin(t) + c1_ * np.cos(t) + (np.sin(t) / 2)   # c2 * sin(t) + c1 * cos(t) + sin(t) / 2
    # At t=0; x = c1
    return y

@njit
def event1(t, y):
    if t > 5.0:
        return 0.0
    else:
        return 1.0

@njit
def event2(t, y):
    if y[0] < 0.0:
        return 0.0
    else:
        return 1.0
event_list = (event1, event2)

# Initial Conditions
# y=0 --> c2 = + 1/2
c2 = 0.5
# x=1 --> c1 = + 1
c1 = 1.0
y0 = np.asarray((0., 1.), dtype=np.float64)
time_span_ = (0., 10.)

In [3]:
from CyRK import pysolve_ivp
from scipy.integrate import solve_ivp

In [6]:
rtols = [1.0e-5, 1.0e-7, 1.0e-9]

integration_method = 'RK45'
t_span = (0., 10.)
t_eval = np.linspace(0.0, 10.0, 100)

for rtol in rtols:
    atol = rtol/10
    
    print("\n RTOL = ", rtol)
    
    scipy_sol   = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method)
    scipy_teval = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval)
    scipy_events = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method, events=event_list)
    
    pysolve_sol   = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, dense_output=True, pass_dy_as_arg=True)
    pysolve_teval = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval, pass_dy_as_arg=True)
    pysolve_events = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, events=event_list, pass_dy_as_arg=True)
    
    print(pysolve_sol.t.shape)
    chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
    chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
    
    print(f"SciPy (sol)\t|\tPySolve (sol)")
    print(f"{chi_sci_sol:0.5e}\t|\t{chi_pysolve_sol:0.5e}")
    
    chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
    chi_pysolve_teval = np.nansum((pysolve_teval.y - correct_answer(pysolve_teval.t, c1, c2))**2 / correct_answer(pysolve_teval.t, c1, c2))
    
    dense_sol = pysolve_sol(t_eval)
    chi_pysolve_dense = np.nansum((dense_sol - correct_answer(t_eval, c1, c2))**2 / correct_answer(t_eval, c1, c2))
    
    print()
    print(f"SciPy (teval)\t|\tPySolve (teval)\t|\tPySolve (dense)")
    print(f"{chi_sci_teval:0.5e}\t|\t{chi_pysolve_teval:0.5e}\t|\t{chi_pysolve_dense:0.5e}")

    print()
    print("Checking Events")
    print(f"SciPy Event 1 len = {scipy_events.t_events[0].size}")
    print(f"SciPy Event 2 len = {scipy_events.t_events[1].size}")
    print(f"pysolve_ivp Event 1 len = {pysolve_events.t_events[0].size}")
    print(f"pysolve_ivp Event 2 len = {pysolve_events.t_events[1].size}")
    
    
#     fig1, ax1 = plt.subplots()
#     ax1.plot(scipy_sol.t, scipy_sol.y[0], c='b')
#     ax1.plot(scipy_sol.t, scipy_sol.y[1], c='r')
#     ax1.set(title="SciPy (sol)")
    
#     fig12, ax12 = plt.subplots()
#     ax12.plot(scipy_teval.t, scipy_teval.y[0], c='b')
#     ax12.plot(scipy_teval.t, scipy_teval.y[1], c='r')
#     ax12.set(title="SciPy (t-eval)")
    
#     fig2, ax2 = plt.subplots()
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[0], c='b')
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[1], c='r')
#     ax2.set(title="PySolve (sol)")
    
#     fig22, ax22 = plt.subplots()
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[0], c='b')
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[1], c='r')
#     ax22.set(title="PySolve (t-eval)")
    
#     fig23, ax23 = plt.subplots()
#     ax23.plot(t_eval, dense_sol[0], c='b')
#     ax23.plot(t_eval, dense_sol[1], c='r')
#     ax23.set(title="PySolve (dense)")
    
#     fig4, ax4 = plt.subplots()
#     ax4.plot(scipy_sol.t, (scipy_sol.y[0] - cyrk_sol[1][0]) / scipy_sol.y[0], c='b')
#     ax4.plot(scipy_sol.t, (scipy_sol.y[1] - cyrk_sol[1][1]) / scipy_sol.y[1], c='r')
#     ax4.set(title="SciPy (sol) - cyrk_ode")
    
#     fig5, ax5 = plt.subplots()
#     ax5.plot(scipy_sol.t, (scipy_sol.y[0] - pysolve_sol.y[0]) / scipy_sol.y[0], c='b')
#     ax5.plot(scipy_sol.t, (scipy_sol.y[1] - pysolve_sol.y[1]) / scipy_sol.y[1], c='r')
#     ax5.scatter(scipy_sol.t, np.zeros_like(scipy_sol.t), c='g', s=1)
#     ax5.scatter(pysolve_sol.t, np.zeros_like(scipy_sol.t), c='purple', s=5)
#     ax5.set(title="SciPy (sol) - pysolve")
    
#     sci_step_size = np.diff(scipy_sol.t)
#     solpy_step_size = np.diff(pysolve_sol.t)
    
#     fig5, ax5 = plt.subplots()
# #     ax5.plot(scipy_sol.t, (scipy_sol.y[0] - pysolve_sol.y[0]) / scipy_sol.y[0], c='b', ls=':')
# #     ax5.plot(scipy_sol.t, (scipy_sol.y[1] - pysolve_sol.y[1]) / scipy_sol.y[1], c='r', ls=':')
#     ax5b = ax5.twinx()
#     ax5b.plot(scipy_sol.t[1:], sci_step_size, c='b', marker='.')
#     ax5b.plot(pysolve_sol.t[1:], solpy_step_size, c='r', marker='.')
#     ax5b.set(ylim=(0.3, 0.4))
#     plt.show()
    
#     break
    
    
    
    
    


 RTOL =  1e-05
(30,)
SciPy (sol)	|	PySolve (sol)
-4.36684e-09	|	-4.36684e-09

SciPy (teval)	|	PySolve (teval)	|	PySolve (dense)
-3.88995e-09	|	-3.88995e-09	|	-3.88995e-09

Checking Events
SciPy Event 1 len = 14
SciPy Event 2 len = 20
pysolve_ivp Event 1 len = 14
pysolve_ivp Event 2 len = 20

 RTOL =  1e-07
(75,)
SciPy (sol)	|	PySolve (sol)
5.11775e-12	|	5.11775e-12

SciPy (teval)	|	PySolve (teval)	|	PySolve (dense)
-6.97744e-13	|	-6.97744e-13	|	-6.97744e-13

Checking Events
SciPy Event 1 len = 38
SciPy Event 2 len = 47
pysolve_ivp Event 1 len = 38
pysolve_ivp Event 2 len = 47

 RTOL =  1e-09
(185,)
SciPy (sol)	|	PySolve (sol)
-8.72413e-17	|	-8.72419e-17

SciPy (teval)	|	PySolve (teval)	|	PySolve (dense)
-5.95678e-17	|	-5.95677e-17	|	-5.95677e-17

Checking Events
SciPy Event 1 len = 93
SciPy Event 2 len = 116
pysolve_ivp Event 1 len = 93
pysolve_ivp Event 2 len = 116


C:\Users\joepr\AppData\Local\Temp\ipykernel_13168\967676045.py:21: RuntimeWarning: invalid value encountered in divide
  chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_13168\967676045.py:22: RuntimeWarning: invalid value encountered in divide
  chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_13168\967676045.py:27: RuntimeWarning: invalid value encountered in divide
  chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_13168\967676045.py:28: RuntimeWarning: invalid value encountered in divide
  chi_pysolve_teval = np.nansum((pysolve_teval.y - correct_answer(pysolve_teval.t, c1, c2))**2 / correct_answer(pysolve_teval.t, c1, c2))
C:\Users